In [1]:
import requests, json
import numpy as np
import pandas as pd

In [12]:
url = requests.get("https://www.materialscloud.org/mcloud/api/v2/discover/sssp/elements/H")
text = url.text
print(type(text))

<class 'str'>


In [13]:
raw_data = json.loads(text)

In [14]:
print(raw_data.keys())

dict_keys(['bands_filenames', 'chessboards_filenames', 'data', 'efficiency_filenames', 'eos_filenames', 'method', 'path', 'precision_filenames', 'url', 'url_root'])


The two libraries (SSSP Precision and SSSP Efficiency) cover for each element either the best in class (among many libraries, or generated by us), or occasionally allow for a compromise (either expensive and accurate, or less expensive and a bit less accurate). 

The convergence pattern of each element includes for each of the considered family (represented by a color):
the total number of electrons in the valence, Z;
delta value (the error in the equation of state compared with all-electron WIEN2k results, developed by Cottenier group), at full converged cutoff;
the largest phonon frequency, ωmax, at the zone boundary (as a number), at full cutoff;

and then as a function of wave function cutoff:
the discrepancy of all phonon frequencies, δω̄, at the zone boundary, with respect to the converged value;
the convergence of the pressure, δVpress, with respect to the converged value;
the convergence of the cohesive energy, δEcoh, with respect to the converged value;
the convergence of the bands structure, η10 and max η10 with respect to the converged value.

The suggested wave function cutoff and dual are indicated (the dual is indicated in parentheses). All normconserving psp have a dual of 4 (i.e. ecutrho=4*ecutwfc); all ultrasoft/PAW have a dual of 8, with the exception of Mn and Fe, that have duals of 12. The tests have been made for PBE pseudopotentials using the Quantum ESPRESSO package. 

In [15]:
raw_data['data']['Y_series'].keys()

dict_keys(['B0', 'B1', 'V0', 'cohesive_energy', 'cohesive_energy_percentage', 'delta', 'phonon_frequencies', 'phonon_frequencies_percentage', 'pressure', 'pressure_percentage'])

In [16]:
set(list(raw_data['data']['Y_series']['delta'].keys()))

{'GBRV_1.2',
 'GBRV_1.4',
 'Goedecker',
 'SG15',
 'pslib.0.3.1_PBE_PAW',
 'pslib.0.3.1_PBE_US',
 'pslib.1.0.0_PBE_PAW',
 'pslib.1.0.0_PBE_US'}

In [17]:
raw_data['data']['Y_series']['delta']['GBRV_1.2']['8.0']['points']

[{'node_uuid': '31e39cca-769f-4cac-9719-4d0d7b18c640', 'value': 1.2006337411},
 {'node_uuid': 'c7ce10f8-2b0b-498e-8e93-d7abf6341d71', 'value': 1.2006337411},
 {'node_uuid': '238f4728-4e09-48e0-b44c-eda9aab92a31', 'value': 1.2006337411},
 {'node_uuid': '99b2c415-a618-4a41-aff8-94a6e8fa6b07', 'value': 1.2006337411},
 {'node_uuid': 'a4f2e1a3-cc80-4f60-947c-5555b81f913f', 'value': 1.2006337411},
 {'node_uuid': '65088bf6-9f0c-4b3a-a9a1-3b11cfb2df98', 'value': 1.2006337411},
 {'node_uuid': 'b1f49e15-e884-420e-986f-aaecdf9069f6', 'value': 1.2006337411},
 {'node_uuid': '044e8d3e-5097-491a-a52b-448b227ace42', 'value': 1.2006337411},
 {'node_uuid': '4a722223-afd0-4b53-b4d8-d4bad2693dfe', 'value': 1.2006337411},
 {'node_uuid': 'fcaf698f-84d6-4223-9b3b-e96041b4df07', 'value': 1.2006337411},
 {'node_uuid': 'c681b340-fb00-4db4-b8fd-4e76e633950d', 'value': 1.2006337411},
 {'node_uuid': '65d84365-5def-489f-a032-d51601be244a', 'value': 1.2006337411},
 {'node_uuid': '8476c718-6dd2-4cd4-8399-3484c0db1ca3

In [18]:
def extract_dE_values(raw_text, potential_key):
    values = []
    raw_data = json.loads(raw_text)
    nodes_list = raw_data['data']['Y_series']['delta'][potential_key]['8.0']['points']
    for k in range(len(nodes_list)):
        node = nodes_list[k]
        values.append(node['value'])
    
    return np.array(values)

In [19]:
potential_key = 'GBRV_1.2'
values = extract_dE_values(text, potential_key)

In [20]:
potential_keys_data = ['pslib.1.0.0_PBE_PAW',
 'RE_Wentz_plus_nitrogen',
 'RE_pslib.1.0.0_PBE_US_plus_nitrogen',
 'GBRV_1.4',
 'Dojo',
 'pslib.0.3.1_PBE_PAW',
 'SG15_1.1',
 'RE_pslib.1.0.0_PBE_PAW_plus_nitrogen',
 'SG15',
 'Goedecker',
 'GBRV_1.2',
 'pslib.0.3.1_PBE_US',
 'GBRV_1.5',
 'pslib.1.0.0_PBE_US']

potential_keys_sssp = ['031PAW',
 '100US',
 'GBRV-1.5',
 'Dojo',
 'Wentzcovitch',
 'THEOS',
 'GBRV-1.2',
 '100PAW',
 'SG15',
 '031US',
 'GBRV-1.4',
 'SG15-1.1']


potential_match_dict = {
    '031PAW' : 'pslib.0.3.1_PBE_PAW',
    '100US' : 'pslib.1.0.0_PBE_US',
    'GBRV-1.5' : 'GBRV_1.5',
    'Dojo' : 'Dojo',
    'Wentzcovitch',
    'THEOS',
    'GBRV-1.2' : 'GBRV_1.2',
    '100PAW' : 'pslib.1.0.0_PBE_PAW',
    'SG15' : 'SG15',
    '031US' : 'pslib.0.3.1_PBE_US',
    'GBRV-1.4': 'GBRV_1.4',
    'SG15-1.1' : 'SG15_1.1'
}

In [21]:
url_table = requests.get("https://archive.materialscloud.org/record/file?record_id=862&filename=SSSP_1.1.2_PBE_efficiency.json&file_id=a5642f40-74af-4073-8dfd-706d2c7fccc2")
text_table = url_table.text

In [22]:
sssp_table = json.loads(text_table)

In [23]:
periodic_table_keys = list(sssp_table.keys())

In [24]:
sssp_dict = {}
for element in periodic_table_keys:
    pot_ssp = sssp_table[element]['pseudopotential']
    pot_data = potential_match_dict[pot]
    sssp_dict[element] = pot_data

In [25]:
Ecutoffs = raw_data['data']['X_series']['Ecutoff']['values']

In [26]:
len(Ecutoffs)

16

In [27]:
def extract_Ecutoff_values(raw_text):
    raw_data = json.loads(raw_text)
    return np.array(raw_data['data']['X_series']['Ecutoff']['values'])

In [28]:
Ecutoffs = extract_Ecutoff_values(text)

In [44]:

base_url = "https://www.materialscloud.org/mcloud/api/v2/discover/sssp/elements/"
potential_keys_data = []
no_delta_elements = []
for element in periodic_table_keys:
    url = requests.get(base_url + element)
    text = url.text
    raw_data = json.loads(text)
    try:
        potential_keys_data.extend(list(raw_data['data']['Y_series']['delta'].keys()))
    except:
        print("*NO DELTA ERROR*")
        print("NO DELTA DATA FOUND FOR ", element, "\n")
        no_delta_elements.append(element)
    try:
        potential_keys_data.extend(list(raw_data['data']['Y_series']['cohesive_energy'].keys()))
        
    except:
        print("*NO COHESIVE ERROR")
potential_keys_data = set(potential_keys_data)


*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Ce 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Dy 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Er 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Eu 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Gd 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Ho 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  La 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Lu 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Nd 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Pm 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Pr 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Sm 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Tb 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Tm 

*NO DELTA ERROR*
NO DELTA DATA FOUND FOR  Yb 



In [ ]:
def extract_element_data(element_key):
    